# Silver Analytics – Fake Job

Este notebook apresenta as análises realizadas sobre a camada **Silver** do projeto,
utilizando dados já limpos e padronizados pelo processo de ETL.

Fonte de dados: PostgreSQL (tabela silver_jobs)


In [1]:
import plotly.express as px
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("Analytics_Silver_Layer") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print("SparkSession recriada com driver PostgreSQL")


:: loading settings :: url = jar:file:/Users/analuizarocha/PycharmProjects/sb2-fake-real-job/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/analuizarocha/.ivy2/cache
The jars for the packages stored in: /Users/analuizarocha/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-afee4364-94ef-4cbb-b682-3734fbb52e5a;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 67ms :: artifacts dl 2ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------

SparkSession recriada com driver PostgreSQL


## 1. Carregamento da base

In [2]:
jdbc_url = "jdbc:postgresql://localhost:5432/fake-real-job"

db_properties = {
    "user": "postgres",
    "password": "senha123",
    "driver": "org.postgresql.Driver"
}

df_silver = spark.read.jdbc(
    url=jdbc_url,
    table="silver_jobs",
    properties=db_properties
)


## 1. Análise de Balanceamento das Classes
### 1.1 Distribuição de Vagas Reais e Fraudulentas

Esta análise tem como objetivo avaliar a proporção entre vagas classificadas como reais e fraudulentas no conjunto de dados.
A verificação do balanceamento entre as classes é fundamental, pois desequilíbrios significativos podem impactar análises estatísticas,
interpretações dos dados e o desempenho de modelos de classificação em etapas futuras.

In [3]:
fraud_dist = (
    df_silver
    .groupBy("is_fake")
    .count()
    .toPandas()
)

fraud_dist["Tipo"] = fraud_dist["is_fake"].map({
    True: "Fraudulenta",
    False: "Real"
})

fig = px.pie(
    fraud_dist,
    names="Tipo",
    values="count",
    hole=0.4
)

fig.update_traces(
    textinfo="percent+label",
    textposition="inside"
)

fig.update_layout(
    legend_title_text="Tipo da Vaga",
    title="Distribuição de Vagas Reais e Fraudulentas",
    title_x=0.5
)

fig.show()


## 2. Análise Geográfica das Vagas Fraudulentas
### 2.1 Distribuição de Vagas Fraudulentas por País

Esta análise investiga a distribuição geográfica das vagas classificadas como fraudulentas, considerando o país de origem das publicações.
O objetivo é identificar concentrações de fraudes em determinadas regiões, permitindo a observação de padrões geográficos
que podem indicar maior incidência de comportamentos suspeitos ou falhas de controle em contextos específicos.

In [4]:
from pyspark.sql.functions import col

fraud_by_country = (
    df_silver
    .filter(col("is_fake") == True)
    .groupBy("country")
    .count()
    .orderBy("count")
    .toPandas()
)

fraud_by_country = fraud_by_country.sort_values("count", ascending=True)

fig = px.bar(
    fraud_by_country,
    x="count",
    y="country",
    orientation="h",
    labels={
        "count": "Quantidade de Vagas Fraudulentas",
        "country": "País"
    },
    text="count",
    color="count",
    color_continuous_scale="Reds"
)

fig.update_traces(
    textposition="outside"
)

fig.update_layout(
    title_x=0.5,
    title="Distribuição de Vagas Fraudulentas por País",
    xaxis_showgrid=True,
    yaxis_showgrid=False,
    coloraxis_showscale=False,
    plot_bgcolor="rgba(0,0,0,0)",
    bargap=0.25
)

fig.show()


## 3. Análise do Regime de Trabalho das Vagas
### 3.1 Distribuição de Vagas Remotas e Presenciais

Esta análise avalia a distribuição das vagas de acordo com o regime de trabalho, classificando-as como remotas ou presenciais.
O objetivo é compreender a predominância de cada modalidade no conjunto de dados da camada Silver,
permitindo identificar tendências relacionadas à forma de contratação e à flexibilidade do trabalho oferecido.

In [5]:
import plotly.express as px

remote_dist = (
    df_silver
    .groupBy("remote")
    .count()
    .toPandas()
)

remote_dist["Tipo"] = remote_dist["remote"].map({
    True: "Remota",
    False: "Presencial"
})

fig = px.bar(
    remote_dist,
    x="Tipo",
    y="count",
    title="Distribuição de Vagas Remotas vs Presenciais",
    text="count",
    color="Tipo",
    color_discrete_map={
        "Remota": "#636EFA",
        "Presencial": "#EF553B"
    },
    labels={
        "count": "Quantidade de Vagas",
        "Tipo": "Tipo da Vaga"
    }
)

fig.update_traces(
    textposition="outside"
)

fig.update_layout(
    title_x=0.5,
    plot_bgcolor="rgba(0,0,0,0)",
    xaxis_showgrid=False,
    yaxis_showgrid=True,
    legend_title_text="Tipo"
)

fig.show()


## 4. Análise de Salários por Tipo de Vaga
### 4.1 Comparação do Salário Médio entre Vagas Reais e Fraudulentas
Esta análise avalia a diferença do salário médio entre vagas reais e fraudulentas.
O objetivo é verificar se há padrões salariais que possam indicar comportamentos atípicos
associados a anúncios fraudulentos.

In [6]:
import plotly.express as px

salary_fraud = (
    df_silver
    .filter(df_silver.salary_avg.isNotNull())
    .groupBy("is_fake")
    .avg("salary_avg")
    .toPandas()
)

salary_fraud["Tipo"] = salary_fraud["is_fake"].map({
    True: "Fraudulenta",
    False: "Real"
})

salary_fraud = salary_fraud.rename(
    columns={"avg(salary_avg)": "Salário Médio"}
)

fig = px.bar(
    salary_fraud,
    y="Tipo",
    x="Salário Médio",
    orientation="h",
    title="Salário Médio por Tipo de Vaga",
    labels={
        "Salário Médio": "Salário Médio ($)",
        "Tipo": ""
    }
)

fig.update_traces(
    textposition="outside",
    marker=dict(color="#6C70F0"),
    width=0.35
)

fig.update_layout(
    title_x=0.5,
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        showgrid=True,
        gridcolor="rgba(255,255,255,0.15)",
        tickprefix="$ ",
        tickformat=",.0f"
    ),
    yaxis=dict(showgrid=False),
    showlegend=False,
    height=300,
    font=dict(size=14)
)

fig.show()


## 5. Análise Temporal das Vagas
### 5.1 Evolução Mensal de Vagas Reais e Fraudulentas

Esta análise avalia a evolução mensal das vagas reais e fraudulentas presentes no dataset.
O objetivo é identificar tendências temporais e padrões de crescimento ou retração na ocorrência de fraudes ao longo do tempo.

In [7]:
import pandas as pd
import plotly.express as px
from pyspark.sql.functions import trunc, col

timeline_month = (
    df_silver
    .withColumn("month", trunc("posting_timestamp", "month"))
    .groupBy("month", "is_fake")
    .count()
    .orderBy("month")
    .toPandas()
)

timeline_month["month"] = pd.to_datetime(timeline_month["month"])

timeline_month["Tipo"] = timeline_month["is_fake"].map({
    True: "Fraudulenta",
    False: "Real"
})

frames = []
unique_months = sorted(timeline_month["month"].unique())

for m in unique_months:
    step = timeline_month[timeline_month["month"] <= m].copy()
    step["animation_frame"] = m.strftime("%Y-%m")
    frames.append(step)

df_final = pd.concat(frames)

fig = px.line(
    df_final,
    x="month",
    y="count",
    color="Tipo",
    animation_frame="animation_frame",
    markers=True,
    title="Evolução Mensal de Vagas Reais e Fraudulentas",
    labels={
        "month": "Mês",
        "count": "Quantidade de Vagas",
        "Tipo": "Classificação"
    },
    range_x=[
        timeline_month["month"].min(),
        timeline_month["month"].max()
    ],
    range_y=[
        0,
        timeline_month["count"].max() * 1.1
    ]
)

fig.update_layout(
    title_x=0.5,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.08)")
)

fig.show()


## 6. Análise dos Motivos de Fraude
### 6.1 Distribuição das Razões Associadas a Vagas Fraudulentas
Esta análise avalia a distribuição das razões de fraude identificadas nas vagas fraudulentas.
O objetivo é destacar os motivos mais recorrentes associados a esse tipo de anúncio.

In [8]:
import plotly.express as px

fraud_reasons = (
    df_silver
    .filter(df_silver.is_fake == True)
    .filter(df_silver.fraud_reason.isNotNull())
    .groupBy("fraud_reason")
    .count()
    .orderBy("count", ascending=False)
    .toPandas()
)

total_vagas = fraud_reasons["count"].sum()

fig = px.pie(
    fraud_reasons,
    names="fraud_reason",
    values="count",
    hole=0.6,
    title="Distribuição das Razões de Fraude"
)

fig.update_traces(
    textposition='outside',
    textinfo='percent+label',
    marker=dict(line=dict(color='#000000', width=1))
)

fig.update_layout(
    title_x=0.5,
    showlegend=False,
    annotations=[dict(text=f'<b>{total_vagas}</b><br>Casos', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

## 7. Análise de Completude dos Dados
7.1 Completude das Colunas na Camada Silver

Esta análise examina o nível de preenchimento das colunas do dataset.
O objetivo é identificar atributos com baixa completude que possam impactar a confiabilidade das análises subsequentes.

In [9]:
from pyspark.sql.functions import col, count, when
import pandas as pd
import plotly.express as px

total_rows = df_silver.count()

completeness = []

for column_name in df_silver.columns:
    non_null_count = df_silver.select(
        count(when(col(column_name).isNotNull(), column_name)).alias("cnt")
    ).collect()[0]["cnt"]

    completeness.append({
        "Coluna": column_name,
        "Preenchimento": round((non_null_count / total_rows) * 100, 1)
    })

completeness_df = pd.DataFrame(completeness)

fig = px.bar(
    completeness_df.sort_values("Preenchimento"),
    x="Preenchimento",
    y="Coluna",
    orientation="h",
    color="Preenchimento",
    color_continuous_scale=["#b30000", "#ffd966", "#006400"],
    range_color=[50, 100],
    title="Completude de Dados por Coluna",
    text="Preenchimento"
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title="Percentual de Preenchimento (%)",
    yaxis_title="",
    coloraxis_colorbar=dict(title="Preenchimento"),
)

fig.show()
